# State-Driven Exposure Bases

## Overview
Exposure bases that track the **actual financial state** of the company rather than artificial growth projections.  Revenue, asset, and equity exposures dynamically adjust claim frequency as the business evolves.

- **Prerequisites**: [core/01_ergodic_foundations](../core/01_ergodic_foundations.ipynb)
- **Estimated runtime**: < 1 minute
- **Audience**: [Developer]

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/optimization'

    os.chdir(NOTEBOOK_DIR)
    if NOTEBOOK_DIR not in sys.path:
        sys.path.append(NOTEBOOK_DIR)

    !pip install ergodic-insurance -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from ergodic_insurance.config import ManufacturerConfig
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.exposure_base import (
    RevenueExposure, AssetExposure, EquityExposure,
)
from ergodic_insurance.loss_distributions import ManufacturingLossGenerator

plt.style.use("seaborn-v0_8-darkgrid")

# Reproducibility
SEED = 42
np.random.seed(SEED)

## 1. Baseline Manufacturer

Create a manufacturer and attach three exposure types.

In [ ]:
config = ManufacturerConfig(
    initial_assets=10_000_000,
    asset_turnover_ratio=1.0,
    base_operating_margin=0.12,
    tax_rate=0.25,
    retention_ratio=0.7,
)
manufacturer = WidgetManufacturer(config)

revenue_exp = RevenueExposure(state_provider=manufacturer)
asset_exp   = AssetExposure(state_provider=manufacturer)
equity_exp  = EquityExposure(state_provider=manufacturer)

print("Initial state:")
print(f"  Assets  : ${float(manufacturer.total_assets):,.0f}")
print(f"  Revenue : ${float(manufacturer.current_revenue):,.0f}")
print(f"  Equity  : ${float(manufacturer.current_equity):,.0f}")
print(f"\nFrequency multipliers (all should be ~1.0):")
print(f"  Revenue : {revenue_exp.get_frequency_multiplier(0):.2f}")
print(f"  Asset   : {asset_exp.get_frequency_multiplier(0):.2f}")
print(f"  Equity  : {equity_exp.get_frequency_multiplier(0):.2f}")

## 2. Exposure Response to a Loss Event

Process a major claim and observe how each exposure type rescales.

In [ ]:
print(f"Before loss: Assets = ${float(manufacturer.total_assets):,.0f}")

company_payment, insurance_payment = manufacturer.process_insurance_claim(
    claim_amount=3_000_000,
    deductible_amount=500_000,
    insurance_limit=10_000_000,
)

print(f"After loss : Assets = ${float(manufacturer.total_assets):,.0f}")
print(f"Company paid: ${float(company_payment):,.0f}")
print(f"\nUpdated multipliers:")
print(f"  Revenue (sqrt scaling)     : {revenue_exp.get_frequency_multiplier(0):.2f}")
print(f"  Asset   (linear scaling)   : {asset_exp.get_frequency_multiplier(0):.2f}")
print(f"  Equity  (cube-root scaling): {equity_exp.get_frequency_multiplier(0):.2f}")

## 3. Scaling Behaviour Comparison

Sweep the asset ratio from 0.2x to 3.0x initial and compare multipliers.

In [ ]:
# Compare how multipliers respond to different asset levels
# We simulate by running years of growth/shrinkage and observing the multiplier changes
ratios = np.linspace(0.2, 3.0, 20)
rev_m, ast_m, eq_m = [], [], []

for r in ratios:
    # Create a fresh manufacturer at a scaled asset level
    scaled_config = ManufacturerConfig(
        initial_assets=int(10_000_000 * r),
        asset_turnover_ratio=1.0,
        base_operating_margin=0.12,
        tax_rate=0.25,
        retention_ratio=0.7,
    )
    scaled_mfg = WidgetManufacturer(scaled_config)

    # Attach exposure bases that reference the original $10M as base
    rev_e = RevenueExposure(state_provider=scaled_mfg)
    ast_e = AssetExposure(state_provider=scaled_mfg)
    eq_e  = EquityExposure(state_provider=scaled_mfg)

    rev_m.append(rev_e.get_frequency_multiplier(0))
    ast_m.append(ast_e.get_frequency_multiplier(0))
    eq_m.append(eq_e.get_frequency_multiplier(0))

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(ratios, rev_m, lw=2, label="Revenue")
ax.plot(ratios, ast_m, lw=2, label="Asset")
ax.plot(ratios, eq_m,  lw=2, label="Equity")
ax.axhline(1, color="gray", ls="--", alpha=0.5)
ax.axvline(1, color="gray", ls="--", alpha=0.5)
ax.set_xlabel("Asset Ratio (scaled / $10M)")
ax.set_ylabel("Frequency Multiplier")
ax.set_title("Exposure Scaling Behaviours")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Full Simulation with State-Driven Exposures

Run a 20-year simulation where claim frequency responds to the manufacturer's evolving state.

In [ ]:
def run_state_driven_simulation(years=20, seed=42):
    np.random.seed(seed)
    cfg = ManufacturerConfig(
        initial_assets=10_000_000,
        asset_turnover_ratio=1.2,
        base_operating_margin=0.15,
        tax_rate=0.25,
        retention_ratio=0.7,
    )
    mfg = WidgetManufacturer(cfg)
    exp = RevenueExposure(state_provider=mfg)

    regular_gen = ManufacturingLossGenerator.create_simple(
        frequency=3.0,
        severity_mean=100_000, severity_std=50_000, seed=seed,
    )
    cat_gen = ManufacturingLossGenerator.create_simple(
        frequency=0.1,
        severity_mean=2_000_000, severity_std=1_000_000, seed=seed + 1,
    )

    history = {k: [] for k in ["year", "assets", "freq_mult", "n_claims", "total_claims"]}
    for yr in range(years):
        history["year"].append(yr)
        history["assets"].append(float(mfg.total_assets))
        history["freq_mult"].append(exp.get_frequency_multiplier(yr))

        revenue = float(mfg.calculate_revenue())
        regular_losses, _ = regular_gen.generate_losses(1.0, revenue)
        cat_losses, _ = cat_gen.generate_losses(1.0, revenue)
        claims = regular_losses + cat_losses

        total = 0
        for c in claims:
            total += float(c.amount)
            if float(c.amount) > 500_000:
                mfg.process_insurance_claim(float(c.amount), 100_000, 5_000_000)
            else:
                mfg.process_insurance_claim(float(c.amount))
        history["n_claims"].append(len(claims))
        history["total_claims"].append(total)

        premium = 200_000 * exp.get_frequency_multiplier(yr)
        mfg.record_insurance_premium(premium)
        mfg.step()
        if hasattr(mfg, 'is_ruined') and mfg.is_ruined:
            print(f"Bankruptcy in year {yr}")
            break
    return pd.DataFrame(history)

sim = run_state_driven_simulation()

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes[0, 0].plot(sim["year"], sim["assets"] / 1e6, lw=2)
axes[0, 0].set_ylabel("Assets ($M)")
axes[0, 0].set_title("Company Assets")

axes[0, 1].plot(sim["year"], sim["freq_mult"], lw=2, color="orange")
axes[0, 1].axhline(1, color="gray", ls="--", alpha=0.5)
axes[0, 1].set_ylabel("Multiplier")
axes[0, 1].set_title("Frequency Adjustment")

axes[1, 0].bar(sim["year"], sim["n_claims"], color="red", alpha=0.7)
axes[1, 0].set_ylabel("Count")
axes[1, 0].set_title("Annual Claim Count")

axes[1, 1].bar(sim["year"], sim["total_claims"] / 1e6, color="darkred", alpha=0.7)
axes[1, 1].set_ylabel("$M")
axes[1, 1].set_title("Annual Claim Amounts")

for ax in axes.flat:
    ax.set_xlabel("Year")
    ax.grid(True, alpha=0.3)

plt.suptitle("State-Driven Exposure Simulation", fontweight="bold")
plt.tight_layout()
plt.show()

print(f"Final assets          : ${sim['assets'].iloc[-1]:,.0f}")
print(f"Avg frequency mult    : {sim['freq_mult'].mean():.2f}")
print(f"Total claims          : {sim['n_claims'].sum()}")
print(f"Avg claims/year       : {sim['n_claims'].mean():.1f}")

## Key Takeaways

- **State-driven exposures** track the real financial state of the company, not static projections.
- Different scaling laws (sqrt, linear, cube-root) suit different risk profiles.
- Frequency multipliers **automatically reduce** when the company shrinks, providing a natural feedback loop.
- The approach aligns with ergodic theory: individual-trajectory dynamics matter more than ensemble averages.

## Next Steps

- [optimization/04_retention_optimization](04_retention_optimization.ipynb) -- retention optimization with market cycles
- [advanced/01_hjb_optimal_control](../advanced/01_hjb_optimal_control.ipynb) -- HJB-based optimal insurance control